In [1]:
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json
DATA_DIR = "../data/"
COSTHRESH = 0.55

In [2]:
EMBEDDING_FILE = '../data/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))


In [3]:
# To create a tree
with open(DATA_DIR+"legal_words.json") as kfile:
    keywords = json.load(kfile)
    
key_index = {}
index_key ={}
keyword_matrix = []
sz = len(embeddings_index.get("court"))
for i,key in enumerate(reversed(sorted(keywords.items(), key=lambda x: x[1]))):
    if key[0] in embeddings_index:
        key_index[key[0]] = i
        index_key[i] = key[0]
        keyword_matrix.append(embeddings_index.get(key[0]))
    else:
        index_key[i] = key[0]
        keyword_matrix.append(np.array([100]*sz))
        print(key[0] + " not found")
    if i > 1000:
        break
keyword_matrix = np.array(keyword_matrix)
tree = KDTree(keyword_matrix)

indlaw not found


In [4]:
def get_similar_words(inword, numwords):
    if inword not in embeddings_index:
        return []
    dist, ind = tree.query(embeddings_index.get(inword).reshape(1,-1),k=numwords) 
    simlist = []
    for hkey in ind[0]:
        simlist.append(index_key[hkey])
    return simlist

def get_keyword(lstkeys):
    actual_word = {}
    for word in lstkeys:
        similar_words = get_similar_words(word,1)
        if len(similar_words)==0:
            continue
        if cosine_similarity(embeddings_index.get(similar_words[0]).reshape(1,-1),embeddings_index.get(word.lower()).reshape(1,-1))[0] > COSTHRESH:
            actual_word[word] = similar_words[0]
    return actual_word

In [5]:
with open(DATA_DIR+"keywords_to_cases.json") as ktc:
    case_keywords = json.load(ktc) 

In [72]:
def get_similar_cases(lstkeys,topn):
    keywords = get_keyword(lstkeys)
    case_score = {}
    for keyword in keywords:
        true_keyword = keywords[keyword]
        if true_keyword in case_keywords:
            for case in case_keywords[true_keyword]:
                if case in case_score:
                    case_score[case].append(keyword)
                else:
                    case_score[case] = [keyword]
    similar_cases = []
    for i,key in enumerate(reversed(sorted(case_score.items(), key=lambda x: len(x[1])))):
        if i == topn:
            break
        similar_cases.append((key[0],key[1]))
    return similar_cases


lstkeys = ["India", "paid", "cusmer"]
get_similar_cases(lstkeys,10)


[('1993_U_141', ['India', 'paid']),
 ('1985_S_32', ['India', 'paid']),
 ('1964_C_73', ['India', 'paid']),
 ('1985_U_39', ['India', 'paid']),
 ('1963_S_20', ['India', 'paid']),
 ('2003_S_469', ['India', 'paid']),
 ('1980_J_7', ['India', 'paid']),
 ('1962_T_69', ['India', 'paid']),
 ('1986_N_11', ['India', 'paid']),
 ('1981_N_12', ['India', 'paid'])]